# Hearth Stone Decker

* https://hsreplay.net/decks/

* 清資料

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import time
import re
import datetime
import sqlite3 as lite
import os
import random
import calendar

url = "https://hsreplay.net/decks/"

# Deck Dictionary

In [2]:
occupation_dict = {
    "Demon Hunter" : "https://hsreplay.net/decks/#playerClasses=DEMONHUNTER",
    "Druid" : "https://hsreplay.net/decks/#playerClasses=DRUID",
    "Hunter" : "https://hsreplay.net/decks/#playerClasses=HUNTER",
    "Mage" : "https://hsreplay.net/decks/#playerClasses=MAGE",
    "Paladin" : "https://hsreplay.net/decks/#playerClasses=PALADIN",
    "Priest" : "https://hsreplay.net/decks/#playerClasses=PRIEST",
    "Rogue" : "https://hsreplay.net/decks/#playerClasses=ROGUE",
    "Shaman" : "https://hsreplay.net/decks/#playerClasses=SHAMAN",
    "Warlock" : "https://hsreplay.net/decks/#playerClasses=WARLOCK",
    "Warrior" : "https://hsreplay.net/decks/#playerClasses=WARRIOR",
}

In [3]:
occupation_alternative_keyword = {
    "Demon Hunter" : "惡魔獵人",
    "Druid" : "德",
    "Hunter" : "獵",
    "Mage" : "法",
    "Paladin" : "聖",
    "Priest" : "牧",
    "Rogue" : "賊",
    "Shaman" : "薩",
    "Warlock" : "術",
    "Warrior" : "戰",
}

# Loading Dictionary & Analysis

In [4]:
# Save
#np.save('Priest_deck_dict.npy', deck_dict)

# Load
#read_dictionary = np.load('DemonHunter_deck_dict.npy',allow_pickle='TRUE').item()

In [5]:
occupation_dict

{'Demon Hunter': 'https://hsreplay.net/decks/#playerClasses=DEMONHUNTER',
 'Druid': 'https://hsreplay.net/decks/#playerClasses=DRUID',
 'Hunter': 'https://hsreplay.net/decks/#playerClasses=HUNTER',
 'Mage': 'https://hsreplay.net/decks/#playerClasses=MAGE',
 'Paladin': 'https://hsreplay.net/decks/#playerClasses=PALADIN',
 'Priest': 'https://hsreplay.net/decks/#playerClasses=PRIEST',
 'Rogue': 'https://hsreplay.net/decks/#playerClasses=ROGUE',
 'Shaman': 'https://hsreplay.net/decks/#playerClasses=SHAMAN',
 'Warlock': 'https://hsreplay.net/decks/#playerClasses=WARLOCK',
 'Warrior': 'https://hsreplay.net/decks/#playerClasses=WARRIOR'}

# Tidy Data

1. get into layer 3 -> get unique card list

2. for each deck, append to the dataframe as a new row

### For all files

In [6]:
import os

unique_card_dict = {}


for occupation in list(occupation_dict.keys()): # for each 職業 （依照職業分類資料夾）
    base_path = f"DB/{occupation}"
    occupation_deck_paths = os.listdir(base_path) # 職業資料夾底下有數個.npy檔，要load進來
    occupation_deck_paths.sort()
    
    for occupation_decks_path in occupation_deck_paths: # 大約一天一個字典檔
        if ".npy" in occupation_decks_path: # 如果是.npy檔，就讀進來
            d_tmp = np.load(f"{base_path}/{occupation_decks_path}", allow_pickle='TRUE').item()
            
            # 在一次爬蟲抓取的dict下
            for deck_key in list(d_tmp.keys()):
                for cardID in list(d_tmp[deck_key]['Deck_Composition'].keys()):
                    #unique_card_list.append(cardID)
                    unique_card_dict[cardID] = d_tmp[deck_key]['Deck_Composition'][cardID]['card_name']

In [7]:
columns = ["Card_Name"]
df_cards = pd.DataFrame.from_dict(unique_card_dict, orient = "index", columns=columns)
df_cards = df_cards.sort_index()
df_cards = df_cards.reset_index()
df_cards.columns = ["Card_ID", "Card_Name"]

In [8]:
df_cards

,Card_ID,Card_Name
0,8,精神控制
1,22,心靈之怒
2,23,末日災厄
3,28,柯爾克隆精英
4,30,思想竊取
...,...,...
968,61842,瑪克希瑪‧布萊頓海默
969,61846,請勿餵食動物
970,61884,舞台鬼母
971,61898,蠕動懼怪


# DataFrame

In [9]:
columns = ["Deck_ID", "Deck_URL", "Deck_Name", "Occupation", 
           "dust_cost", "game_count", "time_duration", "win_rate"]

In [10]:
for Card_ID in df_cards["Card_ID"]:
    columns.append(f"Card_{Card_ID}")
columns[0:10]

['Deck_ID',
 'Deck_URL',
 'Deck_Name',
 'Occupation',
 'dust_cost',
 'game_count',
 'time_duration',
 'win_rate',
 'Card_8',
 'Card_22']

In [43]:
# mother dataframe
df_Decks = pd.DataFrame(columns = columns)

### Append to Mother Dataframe

```python
t0 = datetime.datetime.now()

for occupation in list(occupation_dict.keys()): # for each 職業 （依照職業分類資料夾）
    base_path = f"DB/{occupation}"
    occupation_deck_paths = os.listdir(base_path) # 職業資料夾底下有數個.npy檔，要load進來
    occupation_deck_paths.sort()
    
    for occupation_decks_path in occupation_deck_paths: # 大約一天一個字典檔
        if ".npy" in occupation_decks_path: # 如果是.npy檔，就讀進來
            d_tmp = np.load(f"{base_path}/{occupation_decks_path}", allow_pickle='TRUE').item()
            

    # row by row 地append
    
        df_tmp["Occupation"] = occupation
        for deck_key in list(d_tmp.keys()):

            # Inside first layer
            df_tmp["Deck_ID"] = deck_key
            df_tmp["Deck_URL"] = d_tmp[deck_key]["URL"]
            df_tmp["Deck_Name"] = d_tmp[deck_key]["Deck_Name"]

            df_tmp["dust_cost"] = d_tmp[deck_key]["dust_cost"]
            df_tmp["game_count"] = d_tmp[deck_key]["game_count"]
            df_tmp["time_duration"] = d_tmp[deck_key]["time_duration"]
            df_tmp["win_rate"] = d_tmp[deck_key]["win_rate"]

            # Inside second layer
            for cardID in list(d_tmp[deck_key]['Deck_Composition'].keys()):
                if d_tmp[deck_key]["Deck_Composition"][cardID]["IsDuo"] == 1:
                    df_tmp[f"Card_{cardID}"] = 2
                else:
                    df_tmp[f"Card_{cardID}"] = 1
                    
            # append
            df_Decks = df_Decks.append(df_tmp)
t1 = datetime.datetime.now()
print(t1-t0)
```

### using `df.loc()` method to assign value/string to a single cell

In [45]:
t0 = datetime.datetime.now()

for occupation in list(occupation_dict.keys()): # for each 職業 （依照職業分類資料夾）
    #occupation = list(occupation_dict.keys())[0]

    base_path = f"DB/{occupation}"
    occupation_deck_paths = os.listdir(base_path) # 職業資料夾底下有數個.npy檔，要load進來
    occupation_deck_paths.sort()

    for occupation_decks_path in occupation_deck_paths: # 大約一天一個字典檔
        if ".npy" in occupation_decks_path: # 如果是.npy檔，就讀進來
            d_tmp = np.load(f"{base_path}/{occupation_decks_path}", allow_pickle='TRUE').item()


    # row by row 地append
        # an empty df
        df_tmp = pd.DataFrame(columns = columns)

        df_tmp.loc[0, "Occupation"] = occupation
        for deck_key in list(d_tmp.keys()):

            # Inside first layer
            df_tmp.loc[0, "Deck_ID"] = deck_key
            df_tmp.loc[0, "Deck_URL"] = d_tmp[deck_key]["URL"]
            df_tmp.loc[0, "Deck_Name"] = d_tmp[deck_key]["Deck_Name"]

            df_tmp.loc[0, "dust_cost"] = d_tmp[deck_key]["dust_cost"]
            df_tmp.loc[0, "game_count"] = d_tmp[deck_key]["game_count"]
            df_tmp.loc[0, "time_duration"] = d_tmp[deck_key]["time_duration"]
            df_tmp.loc[0, "win_rate"] = d_tmp[deck_key]["win_rate"]

            # Inside second layer
            for cardID in list(d_tmp[deck_key]['Deck_Composition'].keys()):
                if d_tmp[deck_key]["Deck_Composition"][cardID]["IsDuo"] == 1:
                    df_tmp.loc[0, f"Card_{cardID}"] = 2
                else:
                    df_tmp.loc[0, f"Card_{cardID}"] = 1

            # append
            df_Decks = df_Decks.append(df_tmp)

t1 = datetime.datetime.now()
print(t1-t0)

4:01:24.426482


In [47]:
# check 卡牌張數總和確實是30
df_tmp.iloc[0,8:].sum()

177

In [46]:
df_Decks

,Deck_ID,Deck_URL,Deck_Name,Occupation,dust_cost,game_count,time_duration,win_rate,Card_8,Card_22,...,Card_61690,Card_61699,Card_61700,Card_61702,Card_61839,Card_61842,Card_61846,Card_61884,Card_61898,Card_62049
0,kyCUK1H2Y5wtS52RdFQLXb,https://hsreplay.net/decks/kyCUK1H2Y5wtS52RdFQ...,Aggro Demon Hunter,Demon Hunter,1840,"130,000",5.6 分鐘,61.1%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
0,wKwIEzbviVLvEj6pEP4Coe,https://hsreplay.net/decks/wKwIEzbviVLvEj6pEP4...,Soul Demon Hunter,Demon Hunter,6000,"60,000",7.5 分鐘,59.2%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
0,DqUe87qVw99Dso2poGdBif,https://hsreplay.net/decks/DqUe87qVw99Dso2poGd...,Aggro Demon Hunter,Demon Hunter,3760,"47,000",5.9 分鐘,62.6%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
0,QWLOmOj7uYAGtA5hzIVrTe,https://hsreplay.net/decks/QWLOmOj7uYAGtA5hzIV...,Soul Demon Hunter,Demon Hunter,4360,"16,000",7.3 分鐘,59.4%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
0,4aChHKfoy145otte4P8r1f,https://hsreplay.net/decks/4aChHKfoy145otte4P8...,Token Demon Hunter,Demon Hunter,0,"15,000",7.4 分鐘,50.6%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,UCGbONNBeZEVHf9BD22Nzf,https://hsreplay.net/decks/UCGbONNBeZEVHf9BD22...,炸彈戰,Warrior,10520,250,10.0 分鐘,61.1%,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,xhN5O481q43RMjerBLgIQf,https://hsreplay.net/decks/xhN5O481q43RMjerBLg...,炸彈戰,Warrior,12120,240,8.5 分鐘,57.5%,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0pvRboqwkRXkroebpOSJ8g,https://hsreplay.net/decks/0pvRboqwkRXkroebpOS...,Silas OTK Warrior,Warrior,13460,240,9.6 分鐘,52.3%,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,vp0MsVDE6ye7WhLlZx1org,https://hsreplay.net/decks/vp0MsVDE6ye7WhLlZx1...,ETC Control Warrior,Warrior,11800,200,11.7 分鐘,54.8%,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df_Decks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51961 entries, 0 to 0
Columns: 981 entries, Deck_ID to Card_62049
dtypes: object(981)
memory usage: 389.3+ MB


### Drop Duplicates

In [57]:
df_Decks = df_Decks.drop_duplicates(ignore_index=True, subset=["Deck_ID", "win_rate"])

In [59]:
df_Decks

,Deck_ID,Deck_URL,Deck_Name,Occupation,dust_cost,game_count,time_duration,win_rate,Card_8,Card_22,...,Card_61690,Card_61699,Card_61700,Card_61702,Card_61839,Card_61842,Card_61846,Card_61884,Card_61898,Card_62049
0,kyCUK1H2Y5wtS52RdFQLXb,https://hsreplay.net/decks/kyCUK1H2Y5wtS52RdFQ...,Aggro Demon Hunter,Demon Hunter,1840,"130,000",5.6 分鐘,61.1%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
1,wKwIEzbviVLvEj6pEP4Coe,https://hsreplay.net/decks/wKwIEzbviVLvEj6pEP4...,Soul Demon Hunter,Demon Hunter,6000,"60,000",7.5 分鐘,59.2%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
2,DqUe87qVw99Dso2poGdBif,https://hsreplay.net/decks/DqUe87qVw99Dso2poGd...,Aggro Demon Hunter,Demon Hunter,3760,"47,000",5.9 分鐘,62.6%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
3,QWLOmOj7uYAGtA5hzIVrTe,https://hsreplay.net/decks/QWLOmOj7uYAGtA5hzIV...,Soul Demon Hunter,Demon Hunter,4360,"16,000",7.3 分鐘,59.4%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
4,4aChHKfoy145otte4P8r1f,https://hsreplay.net/decks/4aChHKfoy145otte4P8...,Token Demon Hunter,Demon Hunter,0,"15,000",7.4 分鐘,50.6%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16721,UCGbONNBeZEVHf9BD22Nzf,https://hsreplay.net/decks/UCGbONNBeZEVHf9BD22...,炸彈戰,Warrior,10520,250,10.0 分鐘,61.1%,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16722,xhN5O481q43RMjerBLgIQf,https://hsreplay.net/decks/xhN5O481q43RMjerBLg...,炸彈戰,Warrior,12120,240,8.5 分鐘,57.5%,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16723,0pvRboqwkRXkroebpOSJ8g,https://hsreplay.net/decks/0pvRboqwkRXkroebpOS...,Silas OTK Warrior,Warrior,13460,240,9.6 分鐘,52.3%,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16724,vp0MsVDE6ye7WhLlZx1org,https://hsreplay.net/decks/vp0MsVDE6ye7WhLlZx1...,ETC Control Warrior,Warrior,11800,200,11.7 分鐘,54.8%,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Card_1234中的`NaN`補0

In [71]:
# 觀察有多少na
np.sum(df_Decks.isna())

Deck_ID           0
Deck_URL          0
Deck_Name         0
Occupation        0
dust_cost         0
              ...  
Card_61842    15667
Card_61846    15501
Card_61884    15817
Card_61898     6740
Card_62049    15199
Length: 981, dtype: int64

In [73]:
df_Decks = df_Decks.fillna(0)

In [74]:
df_Decks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16726 entries, 0 to 16725
Columns: 981 entries, Deck_ID to Card_62049
dtypes: int64(973), object(8)
memory usage: 125.2+ MB


### 處理時間格式

In [78]:
df_Decks["time_duration"]

0         5.6 分鐘
1         7.5 分鐘
2         5.9 分鐘
3         7.3 分鐘
4         7.4 分鐘
          ...   
16721    10.0 分鐘
16722     8.5 分鐘
16723     9.6 分鐘
16724    11.7 分鐘
16725    11.7 分鐘
Name: time_duration, Length: 16726, dtype: object

In [86]:
time_duration_re = []
for t in list(df_Decks["time_duration"]):
    time_duration_re.append(float(t.split()[0]))

In [88]:
df_Decks["time_duration_RE"] = time_duration_re

### 處理win_rate

In [92]:
win_rate_re = []
for t in list(df_Decks["win_rate"]):
    win_rate_re.append(float(t.split("%")[0]))

In [94]:
df_Decks["win_rate_RE"] = win_rate_re

### 建立職業的dummies

In [99]:
dummy = pd.get_dummies(df_Decks["Occupation"])

In [105]:
df_Decks = pd.concat([df_Decks, dummy], axis=1)

In [106]:
df_Decks.columns

Index(['Deck_ID', 'Deck_URL', 'Deck_Name', 'Occupation', 'dust_cost',
       'game_count', 'time_duration', 'win_rate', 'Card_8', 'Card_22',
       ...
       'Demon Hunter', 'Druid', 'Hunter', 'Mage', 'Paladin', 'Priest', 'Rogue',
       'Shaman', 'Warlock', 'Warrior'],
      dtype='object', length=993)

### 去除game_count中的,

In [109]:
game_count_re = []
for t in list(df_Decks["game_count"]):
    game_count_re.append(int("".join(t.split(","))))
df_Decks["game_count_RE"] = game_count_re

### 將所有數值轉為float

In [113]:
# sample dataframe 
df_Decks
# using dictionary to convert specific columns 
convert_dict = {'dust_cost': int,
                'game_count_RE': int,
               }   
df_Decks = df_Decks.astype(convert_dict)
print(df_Decks.dtypes)

Deck_ID          object
Deck_URL         object
Deck_Name        object
Occupation       object
dust_cost         int64
                  ...  
Rogue             uint8
Shaman            uint8
Warlock           uint8
Warrior           uint8
game_count_RE     int64
Length: 994, dtype: object


In [114]:
df_Decks

,Deck_ID,Deck_URL,Deck_Name,Occupation,dust_cost,game_count,time_duration,win_rate,Card_8,Card_22,...,Druid,Hunter,Mage,Paladin,Priest,Rogue,Shaman,Warlock,Warrior,game_count_RE
0,kyCUK1H2Y5wtS52RdFQLXb,https://hsreplay.net/decks/kyCUK1H2Y5wtS52RdFQ...,Aggro Demon Hunter,Demon Hunter,1840,"130,000",5.6 分鐘,61.1%,0,0,...,0,0,0,0,0,0,0,0,0,130000
1,wKwIEzbviVLvEj6pEP4Coe,https://hsreplay.net/decks/wKwIEzbviVLvEj6pEP4...,Soul Demon Hunter,Demon Hunter,6000,"60,000",7.5 分鐘,59.2%,0,0,...,0,0,0,0,0,0,0,0,0,60000
2,DqUe87qVw99Dso2poGdBif,https://hsreplay.net/decks/DqUe87qVw99Dso2poGd...,Aggro Demon Hunter,Demon Hunter,3760,"47,000",5.9 分鐘,62.6%,0,0,...,0,0,0,0,0,0,0,0,0,47000
3,QWLOmOj7uYAGtA5hzIVrTe,https://hsreplay.net/decks/QWLOmOj7uYAGtA5hzIV...,Soul Demon Hunter,Demon Hunter,4360,"16,000",7.3 分鐘,59.4%,0,0,...,0,0,0,0,0,0,0,0,0,16000
4,4aChHKfoy145otte4P8r1f,https://hsreplay.net/decks/4aChHKfoy145otte4P8...,Token Demon Hunter,Demon Hunter,0,"15,000",7.4 分鐘,50.6%,0,0,...,0,0,0,0,0,0,0,0,0,15000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16721,UCGbONNBeZEVHf9BD22Nzf,https://hsreplay.net/decks/UCGbONNBeZEVHf9BD22...,炸彈戰,Warrior,10520,250,10.0 分鐘,61.1%,0,2,...,0,0,0,0,0,0,0,0,1,250
16722,xhN5O481q43RMjerBLgIQf,https://hsreplay.net/decks/xhN5O481q43RMjerBLg...,炸彈戰,Warrior,12120,240,8.5 分鐘,57.5%,0,2,...,0,0,0,0,0,0,0,0,1,240
16723,0pvRboqwkRXkroebpOSJ8g,https://hsreplay.net/decks/0pvRboqwkRXkroebpOS...,Silas OTK Warrior,Warrior,13460,240,9.6 分鐘,52.3%,0,2,...,0,0,0,0,0,0,0,0,1,240
16724,vp0MsVDE6ye7WhLlZx1org,https://hsreplay.net/decks/vp0MsVDE6ye7WhLlZx1...,ETC Control Warrior,Warrior,11800,200,11.7 分鐘,54.8%,0,2,...,0,0,0,0,0,0,0,0,1,200


### Select Columns

In [115]:
df_Decks.columns

Index(['Deck_ID', 'Deck_URL', 'Deck_Name', 'Occupation', 'dust_cost',
       'game_count', 'time_duration', 'win_rate', 'Card_8', 'Card_22',
       ...
       'Druid', 'Hunter', 'Mage', 'Paladin', 'Priest', 'Rogue', 'Shaman',
       'Warlock', 'Warrior', 'game_count_RE'],
      dtype='object', length=994)

In [ ]:
df_Decks = df_Decks.drop(["game_count", "time_duration", "win_rate", "Occupation"], axis = 1)

#### Re-order Columns

In [145]:
df_Decks = df_Decks.reindex(sorted(df_Decks.columns, reverse=True), axis=1)

# Save DataFrame

In [150]:
df_Decks.to_csv("DB/df_Decks_tidy.csv")

In [153]:
np.save("DB/df_Decks_tidy.npy", df_Decks, allow_pickle=True)

In [154]:
df_Decks.to_excel("DB/df_Decks_tidy.xlsx")

In [156]:
df_cards.to_csv("DB/df_cards.csv")